In [ ]:
%load_ext autoreload
%aimport -pandas, -numpy
%autoreload 2
import pandas as pd
import numpy as np
import arrow
from dfsm import FSMOperator

In [ ]:
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(0)

In [ ]:
dval = dmyplant2.Validation.load_def_csv("input2.csv")
i = 5
vale = dval.iloc[i]
#[['serialNumber','Validation Engine','val start']]
vale

In [ ]:
sn = 1243362
e=dmyplant2.Engine.from_sn(mp,sn, valstart='2020-09-07')
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
e.get_dataItems(['Count_OpHour','Count_Start'])

In [ ]:
e.fetch_dataItems(1645281269000,['Count_OpHour','Count_Start'])

In [ ]:
id = 157701
ts = int(pd.to_datetime('2022-01-10').timestamp()*1e3)
print(ts)
#ts = 1645281269000
erg = mp._fetch_oph_starts(ts,id)
erg


In [ ]:
mp.historical_dataItem(157701, 161, 1641772800000).get('value', None)

In [ ]:
mp.historical_dataItem(157701, 179, 1641772800000).get('value', None)

In [ ]:
for d in e.properties:
        print(d, e[d])

In [ ]:
res = e.get_keyItem_data('dataItems','Count_OpHour')
{ res['id'] : [res['name'],res['unit']] }

In [ ]:
e.get_keyItem('Count_OpHour')

In [ ]:
e.get_keyId('Count_OpHour')

In [ ]:
datal = ['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif']
dat = e.get_dataItems(['Count_OpHour','Count_Start','startup_counter'])
dat

In [ ]:
#def _batch_hist_dataItems(self, itemIds={161: ['CountOph', 'h']}, p_limit=None, p_from=None, p_to=None, timeCycle=3600,
#                              assetType='J-Engine', includeMinMax='false', forceDownSampling='false'):
e._batch_hist_dataItems(dat, p_from=1645281269000, p_to=1645281269000).iloc[0]

In [ ]:
def fetch_dataItems(e, items, ts):
    itemIds = e.get_dataItems(items)
    tdj = ','.join([str(s) for s in itemIds])
    url=fr"/asset/{e['id']}/history/batchdata?assetType=J-Engine&from={ts}&to={ts}&dataItemIds={tdj}&timeCycle=30"
    data =  e._mp.fetchdata(url)
    # # restructure data to dict
    # ds = {}
    # ds['labels'] = ['timestamp'] + [itemIds[x][0] for x in data['columns'][1]]
    # ds['data'] = [[r[0]] + [rr[0] for rr in r[1]] for r in [data['data'][0]]]
    # ds['index'] = [r[0] for r in data['data'][0] ]
    # # import to Pandas DataFrame
    # df = pd.DataFrame(ds['data'], columns=ds['labels'])
    # return df
    return data

mdata = fetch_dataItems(e, ['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif'], 1645281269000)

In [ ]:
itemIds = e.get_dataItems(['Count_OpHour','Power_PowerAct','Hyd_PressCrankCase','Hyd_PressOilDif'])
data = {
    'columns': ['time', [161, 217, 16546, 102]],
    'data': 
    [
        [
            1645281269000,
            [
                [71782.0],
                [-15.90],
                [0.5725],
                [2009.0]
            ]
        ],
        [
            1645281269001,
            [
                [71783.0],
                [-15.91],
                [0.5726],
                [2009.5]
            ]
        ]
    ]
}
data

In [ ]:
d = {}
ldata = [data['data'][0]]
for i, key in enumerate(data['columns']):
    if isinstance(key, str):
        d[key] = [value[i] for value in ldata]
    if isinstance(key, list):
        for j, skey in enumerate(key):
            d[itemIds[skey][0]] = [ value[i][j][0] for value in ldata ]
from pprint import pprint as pp
pd.DataFrame(d)

In [ ]:
d = []
from pprint import pprint as pp
for i, key in enumerate(data['columns']):
    d.append({key: [value[i] for value in data['data']]} if isinstance(key, str) else \
         {itemIds[skey][0]:[value[i][j][0] for value in data['data']] for j,skey in enumerate(key)})
{**d[0] ,**d[1]}

In [ ]:
# needs python 3.9.x
# d =  [{key: [value[i] for value in [data['data'][0]]]} if isinstance(key, str) \
#     else {itemIds[skey][0]:[value[i][j][0] for value in [data['data'][0]]] \
#         for j,skey in enumerate(key)} \
#             for i, key in enumerate(data['columns'])]
# pd.DataFrame([d[0]|dd for dd in d[1:]][0])

In [ ]:
# dtime = {'timestamp' : [value[0] for value in [data['data'][0]]]}
# ddata = {itemIds[skey][0]:[value[1][j][0] for value in [data['data'][0]]] for j,skey in enumerate(data['columns'][1])}
# d = dtime|ddata
# d

In [ ]:
e.fetch_dataItems(1645281269000,['Count_OpHour','Various_Values_SpeedAct','Power_PowerAct','Power_SetPower','Hyd_PressCrankCase','Hyd_PressCoolWat','rP_Ramp_Set'])

In [ ]:
mp.fetch_dataItems(1645281269000,e['id'],['Count_OpHour','Various_Values_SpeedAct','Power_PowerAct','Power_SetPower','Hyd_PressCrankCase','Hyd_PressCoolWat','rP_Ramp_Set'])

In [ ]:
e.asset['validation']

In [ ]:
e['Power_PowerNominal']

In [ ]:
#mp._fetch_installed_base()
e.lookup_Installed_Fleet(mp,e._sn)

In [ ]:

e.get_messages()

In [ ]:
for k in sorted(e.properties.keys()):
    print(k)

In [ ]:
100 / e['rP_Ramp_Set']

In [ ]:
'Ramp' in 'rP_Ramp_Act'

In [ ]:
for i, k in enumerate(sorted(e.properties.keys())):
    print(f"{i:03} {k}: {e[k]}")

In [ ]:
for i, k in enumerate(sorted(e.dataItems.keys())):
    if not k.startswith('Para_') and e[k] != None:
        #if k.startswith('Knock_KLS98_Knock'):
            print(f"{i:04} {k}: {e[k]}")

In [ ]:
for k in sorted(e.dataItems.keys()):
    if 'Ramp' in k:
        print(k, e[k])

In [ ]:
for i, k in enumerate(sorted(e.dataItems.keys())):
    if not k.startswith('Para_') and e[k] != None:
        if k[-2:] in [f"{i:02}" for i in range(1,25)]:
            print(f"{i:04} {k}: {e[k]}")